In [1]:
from __future__ import absolute_import
from datetime import datetime, timedelta, date
import hashlib
import requests
import os
import sys
from os.path import *
import urllib
import xlrd
sys.path.insert(0, '../')
try:
    from bs4 import (
        BeautifulSoup, Tag, Comment, ProcessingInstruction, NavigableString,
        Declaration, Doctype)
    _DECLARATION_OR_DOCTYPE = (Declaration, Doctype)
except ImportError:
    from BeautifulSoup import (
        BeautifulSoup, Tag, Comment, ProcessingInstruction, NavigableString,
        Declaration)
    _DECLARATION_OR_DOCTYPE = Declaration
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as cond
from selenium.common.exceptions import NoAlertPresentException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from pyvirtualdisplay.display import Display
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time #using this library to pause or "sleep" program

In [ ]:
#Steps (broad overview)
#1. login to gmail (use skytruth email)
#2. open correct email (Echo Notify - need to make an account)
#3. Scrape email (get more into this later - first let's focus on basics)

In [52]:
#Open gmail - set up driver, navigate to website 
driver=webdriver.Firefox(executable_path=f'C:\Drivers\FirefoxDriver\geckodriver.exe')
driver.implicitly_wait(60) # seconds
driver.maximize_window()
driver.get('https://mail.google.com')
#driver.get('https://echo.epa.gov/detailed-facility-report?fid=110020506460') #for testing purposes - will remove later. goes straight to webpage (this is what I had to do when I was having the issue with the link in the email taking me to a blank screen - the webscraper code still works on this webpage)

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_31476\2567143633.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Firefox(executable_path=f'C:\Drivers\FirefoxDriver\geckodriver.exe')


In [5]:
#Login to gmail - enter username, press next, enter password

In [53]:
#enter username
driver.find_element(By.ID, "identifierId").send_keys("tatiana@skytruth.org") #enter your username 
#click the 'next' button
driver.find_element(By.ID, "identifierNext").click()

In [54]:
#enter password
driver.find_element(By.NAME, "Passwd").send_keys("#########") #enter your password
#click the 'next' button 
driver.find_element(By.ID, "passwordNext").click()

In [ ]:
#may need to manually enter a verification code and press "next" button to complete login.

In [ ]:
#need to add a "wait" here, because usually get sent an email about a security alert, and then the following code has errors executing.

In [56]:
#Will eventually need to edit this to comb through  multiple pages if necesssary, if desired email is not on first page.
email_titles=driver.find_elements(By.XPATH, "//span[@class='bog']") #create list; all email subjects are of class 'bog'
for title in email_titles: #now comb through list; if email subject matches our desired email subject, click the email.
    if title.text == "EPA ECHO Notify: Air Results for SEP 24, 2022 - OCT 01, 2022": #if the subject in the list matches our desired email subject...
        subject=title.text
        title.click()
        print("Email clicked:", subject)
        
    

Email clicked: EPA ECHO Notify: Air Results for SEP 24, 2022 - OCT 01, 2022


In [57]:
#click on link
#Notes: The <a> tag defines a hyperlink, which is used to link from one page to another.
#The most important attribute of the <a> element is the href attribute, which indicates the link's destination.

#will need to find a way to automate this process rather than having to manually enter the link text. 
driver.find_element(By.LINK_TEXT,'CABAA00006013A0010').click()

In [61]:
#test 
#tester=driver.find_element(By.XPATH, "/html[1]/body[1]/section[1]/div[1]/div[2]/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/main[1]/div[1]/section[3]/section[1]/div[1]/section[1]/section[2]/div[1]/div[1]/h4[1]").text
#/html[1]/body[1]/section[1]/div[1]/div[2]/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/main[1]/div[1]/section[3]/section[1]/div[1]/section[1]/section[2]/div[1]/div[1]/h4[1]
#print(tester)

NoSuchElementException: Message: Unable to locate element: /html[1]/body[1]/section[1]/div[1]/div[2]/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/main[1]/div[1]/section[3]/section[1]/div[1]/section[1]/section[2]/div[1]/div[1]/h4[1]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.jsm:12:1
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:404:5
element.find/</<@chrome://remote/content/marionette/element.js:291:16


In [50]:
#test
#facility_address=driver.find_element(By.XPATH, "//section[@class='echo-subsection echo-grid-two-thirds']/div[@class='echo-grid-one-half']/div/h4").text
#print(facility_address)

CHEVRON REFINERY RICHMOND


In [18]:
#the following code is no longer working on the webpage
#the url of the tester facility has changed from https://echo.epa.gov/detailed-facility-report?fid=110020506460 to https://echo.epa.gov/detailed-facility-report?fid=CABAA00006013A0010
#the following code works on the former link, but not the current link.
#the hyperlink provided in the email now leads to the latter link where the scraper is not working.

#facility_address=driver.find_element(By.XPATH, "//section[@class='echo-subsection echo-grid-two-thirds']/div[@class='echo-grid-one-half']/div[@class='mobile-hide']").text
#print(facility_address)

#frs_id=driver.find_element(By.XPATH, "//section[@class='echo-subsection echo-grid-two-thirds']/div[@class='echo-grid-one-half']/dl/div/dd").text
#print("FRS ID: "+ frs_id)

#epa_region=driver.find_element(By.XPATH, "//section[@class='echo-subsection echo-grid-two-thirds']/div[@class='echo-grid-one-half']/dl/div[2]/dd").text
#print("EPA Region: "+ epa_region)

#lat=driver.find_element(By.XPATH, "//section[@class='echo-subsection echo-grid-two-thirds']/div[@class='echo-grid-one-half']/dl/div[3]/dd").text
#print("Latitude: "+ lat)

#long=driver.find_element(By.XPATH, "//section[@class='echo-subsection echo-grid-two-thirds']/div[@class='echo-grid-one-half']/dl/div[4]/dd").text
#print("Longitude: "+ long)

#locational_data_source=driver.find_element(By.XPATH, "//section[@class='echo-subsection echo-grid-two-thirds']/div[@class='echo-grid-one-half']/dl/div[5]/dd").text
#print("Locational Data Source: "+ locational_data_source)

#industries=driver.find_element(By.XPATH, "//section[@class='echo-subsection echo-grid-two-thirds']/div[@class='echo-grid-one-half']/dl/div[6]/dd").text
#print("Industries: "+ industries)

#indian_country_y_or_n=driver.find_element(By.XPATH, "//section[@class='echo-subsection echo-grid-two-thirds']/div[@class='echo-grid-one-half']/dl/div[7]/dd").text
#print("Indian country: "+ indian_country_y_or_n)

NoSuchElementException: Message: Unable to locate element: //section[@class='echo-subsection echo-grid-two-thirds']/div[@class='echo-grid-one-half']/div[@class='mobile-hide']
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.jsm:12:1
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:404:5
element.find/</<@chrome://remote/content/marionette/element.js:291:16
